##### Import the libraries

In [1]:
import os
import json
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from torchvision.ops import box_iou
from torchvision.ops import nms
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.metrics import precision_recall_fscore_support
import rich
import numpy as np
from tqdm import tqdm
import shutil

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

c:\Users\mohamad\AppData\Local\Programs\Python\Python310\lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.4' (you have '2.0.2'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


##### Data Augmentation

In [ ]:
class FacadeDataset(Dataset):
    def __init__(self, image_dir, annotations_dir, transform=None):
        self.image_dir = image_dir
        self.annotations_dir = annotations_dir
        self.transform = transform
        self.images = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def load_annotations(self, annotation_file):
        with open(annotation_file) as f:
            data = json.load(f)
        
        boxes = []
        labels = []
        
        for shape in data["shapes"]:
            if shape["label"] == "window":
                points = shape["points"]
                
                # Calculate bounding box from polygon points
                x_coords = [p[0] for p in points]
                y_coords = [p[1] for p in points]
                x_min = min(x_coords)
                y_min = min(y_coords)
                x_max = max(x_coords)
                y_max = max(y_coords)
                
                # Bounding box format: [x_min, y_min, width, height]
                box = [x_min, y_min, x_max - x_min, y_max - y_min]
                boxes.append(box)
                labels.append(1)  # Use 1 for "window" category label

        return {"boxes": boxes, "labels": labels}

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        annotation_path = os.path.join(self.annotations_dir, f"{os.path.splitext(img_name)[0]}.json")

        image = Image.open(img_path).convert("RGB")
        annotations = self.load_annotations(annotation_path)

        # Convert bounding boxes and labels to tensors
        boxes = torch.tensor(annotations["boxes"], dtype=torch.float32)
        labels = torch.tensor(annotations["labels"], dtype=torch.int64)

        if self.transform:  
            image = self.transform(image)

        target = {"boxes": boxes, "labels": labels}
        return image, target


# Define transformations
transform = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Directories for training
train_image_dir = os.path.join("ZJU_dataset", "images")
train_annotations_dir = os.path.join("ZJU_dataset", "annotation")

# Create dataset
initial_dataset = FacadeDataset(train_image_dir, train_annotations_dir, transform=transform)

# print("Dataset :", dataset[0])

print("Dataset length:", len(initial_dataset))

In [ ]:
# Define augmentation pipeline
augmentations = A.Compose([
    A.HorizontalFlip(p=1.0),
    # A.RandomBrightnessContrast(p=0.2),
    # A.Rotate(limit=10, p=0.3, border_mode=cv2.BORDER_CONSTANT),  # Ensure rotated bbox stays valid
    # A.Blur(p=0.1),
    # A.MotionBlur(p=0.1),
    # A.GaussNoise(p=0.1),
    # A.Resize(256, 256),  # Resize for consistency
    ToTensorV2()
], bbox_params=A.BboxParams(format="pascal_voc", label_fields=["labels"], check_each_transform=True, min_visibility=0.3))

# Function to apply augmentation
def augment_and_save(dataset, save_dir, aug_count=2):
    image_save_dir = os.path.join(save_dir, "images")
    annotation_save_dir = os.path.join(save_dir, "annotation")
    os.makedirs(image_save_dir, exist_ok=True)
    os.makedirs(annotation_save_dir, exist_ok=True)

    for i in range(len(dataset)):
        image, target = dataset[i]
        image_np = np.array(image.permute(1, 2, 0))  # Convert to NumPy array
        boxes = target["boxes"].tolist()
        labels = target["labels"].tolist()

        # Convert from COCO format [x_min, y_min, width, height] → Pascal VOC [x_min, y_min, x_max, y_max]
        boxes = [[b[0], b[1], b[0] + b[2], b[1] + b[3]] for b in boxes]

        for j in range(aug_count):
            augmented = augmentations(image=image_np, bboxes=boxes, labels=labels)
            aug_image = augmented["image"]
            aug_boxes = augmented["bboxes"]

            # Convert back to COCO format if needed
            aug_boxes = [[b[0], b[1], b[2] - b[0], b[3] - b[1]] for b in aug_boxes]

            # Skip if all bounding boxes are removed
            if len(aug_boxes) == 0:
                continue

            # Convert back to PIL Image
            aug_image_pil = transforms.ToPILImage()(aug_image)

            # Save augmented image
            new_filename = f"{i}_aug{j}.jpg"
            aug_image_pil.save(os.path.join(image_save_dir, new_filename))

            # Save augmented annotations
            aug_annotation = {
                "imagePath": new_filename,  # Add image filename to JSON
                "shapes": [
                    {"label": "window", "points": [[x, y] for x, y in [[b[0], b[1]], [b[0] + b[2], b[1] + b[3]]]]}
                    for b in aug_boxes
                ]
            }
            with open(os.path.join(annotation_save_dir, f"{i}_aug{j}.json"), "w") as f:
                json.dump(aug_annotation, f)

    print("Augmentation completed and saved!")

# Run augmentation process
augment_and_save(initial_dataset, save_dir="ZJU_dataset_augmented", aug_count=1)

In [ ]:
# Define dataset paths
base_dir = "ZJU_dataset_full"
full_image_dir = os.path.join(base_dir, "images")
full_annotations_dir = os.path.join(base_dir, "annotation")

# Create new dataset directories
os.makedirs(full_image_dir, exist_ok=True)
os.makedirs(full_annotations_dir, exist_ok=True)

# Function to copy files from source to destination
def copy_files(src_dir, dst_dir):
    if os.path.exists(src_dir):
        for file_name in os.listdir(src_dir):
            src_path = os.path.join(src_dir, file_name)
            dst_path = os.path.join(dst_dir, file_name)
            shutil.copy(src_path, dst_path)

# Copy images and annotations from both datasets
copy_files("ZJU_dataset/images", full_image_dir)
copy_files("ZJU_dataset_augmented/images", full_image_dir)
copy_files("ZJU_dataset/annotation", full_annotations_dir)
copy_files("ZJU_dataset_augmented/annotation", full_annotations_dir)

print("Dataset merging completed successfully!")

##### Draw segmentations on the images to create masks

In [ ]:
# Define the directories
image_directory = os.path.join("ZJU_dataset_full", "images")
input_directory = os.path.join("ZJU_dataset_full", "annotation")
output_directory = os.path.join("ZJU_dataset_full", "masks")

# Create the masks directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over each JSON file in the input directory
for json_file in os.listdir(input_directory):
    if not json_file.endswith('.json'):
        continue

    json_path = os.path.join(input_directory, json_file)

    # Load the JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Get the image details from the JSON file
    image_filename = data['imagePath']
    image_path = os.path.join(image_directory, image_filename)

    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Image {image_path} does not exist.")
        continue

    # Load the image to get its dimensions
    image = Image.open(image_path)
    width, height = image.size

    # Create a blank mask
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)

    for shape in data['shapes']:
        if shape['label'] == 'window':  # Check if the label is 'window'
            points = shape['points']

            # Check if only two points are provided (top-left and bottom-right)
            if len(points) == 2:
                (x1, y1), (x2, y2) = points
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Convert to four-point rectangle
                polygon = [(x1, y1), (x1, y2), (x2, y2), (x2, y1)]
            else:
                # If already four points, just convert them to integers
                polygon = [((x), (y)) for x, y in points]

            draw.polygon(polygon, outline=50, fill=255)


    # Save the mask
    mask_filename = f"mask_{image_filename.replace('.jpg', '.png')}"
    mask_path = os.path.join(output_directory, mask_filename)
    mask.save(mask_path)
    print(f"Saved mask for {image_filename} to {mask_path}")

##### create the masks for the evalution dataset for the semantic segmentation

In [ ]:
import os
import json
from PIL import Image, ImageDraw

# Define the directories
image_directory = "Evaluation_dataset"
annotation_file = "Evaluation_dataset/merged.json"
output_directory = "Evaluation_dataset/masks"

# Create the masks directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Load the JSON file
with open(annotation_file, 'r') as f:
    data = json.load(f)

# Create a mapping of image_id to filename and dimensions
image_info = {img['id']: (img['file_name'], img['width'], img['height']) for img in data['images']}

# Group annotations by image_id
annotations_by_image = {}
for annotation in data['annotations']:
    image_id = annotation['image_id']
    if image_id not in annotations_by_image:
        annotations_by_image[image_id] = []
    annotations_by_image[image_id].append(annotation)

# Process each image
for image_id, annotations in annotations_by_image.items():
    # Get image filename and dimensions
    if image_id not in image_info:
        continue

    image_filename, width, height = image_info[image_id]
    image_path = os.path.join(image_directory, image_filename)

    # Check if the image exists
    if not os.path.exists(image_path):
        print(f"Image {image_path} does not exist.")
        continue

    # Create a blank mask
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)

    # Draw all window annotations on the mask
    for annotation in annotations:
        if 'segmentation' in annotation and len(annotation['segmentation']) > 0:
            for segment in annotation['segmentation']:
                polygon = [(segment[i], segment[i + 1]) for i in range(0, len(segment), 2)]
                draw.polygon(polygon, outline=50, fill=255)

    # Save the mask
    mask_filename = f"mask_{image_filename.replace('.jpg', '.png')}"
    mask_path = os.path.join(output_directory, mask_filename)
    mask.save(mask_path)
    # print(f"Saved mask for {image_filename} with {len(annotations)} windows to {mask_path}")

##### split the dataset & create annotation based on yolo requirements

In [ ]:
# Paths for input data
image_dir = os.path.join("ZJU_dataset", "images")
annotation_dir = os.path.join("ZJU_dataset", "annotation")

# Paths for YOLO-formatted dataset
output_dir = "datasets"
train_images_dir = os.path.join(output_dir, "train", "images")
train_labels_dir = os.path.join(output_dir, "train", "labels")
val_images_dir = os.path.join(output_dir, "val", "images")
val_labels_dir = os.path.join(output_dir, "val", "labels")

# Create output directories
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Function to convert annotations to YOLO format
def convert_to_yolo_format(annotation_path, image_width, image_height):
    with open(annotation_path) as f:
        data = json.load(f)

    yolo_annotations = []
    for shape in data["shapes"]:
        if shape["label"] == "window":  # Filter for "window" objects
            points = shape["points"]
            x_coords = [p[0] for p in points]
            y_coords = [p[1] for p in points]
            x_min = min(x_coords)
            y_min = min(y_coords)
            x_max = max(x_coords)
            y_max = max(y_coords)

            # Convert to YOLO format [class_id, center_x, center_y, width, height]
            center_x = ((x_min + x_max) / 2) / image_width
            center_y = ((y_min + y_max) / 2) / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height

            yolo_annotations.append(f"0 {center_x} {center_y} {width} {height}")

    return yolo_annotations

# Load all image filenames and split into train/val sets
image_filenames = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
train_files, val_files = train_test_split(image_filenames, test_size=0.2, random_state=42)

# Process and save train and val data
for dataset, image_files, images_dir, labels_dir in [
    ("train", train_files, train_images_dir, train_labels_dir),
    ("val", val_files, val_images_dir, val_labels_dir)
]:
    for img_file in image_files:
        img_path = os.path.join(image_dir, img_file)
        annotation_path = os.path.join(annotation_dir, f"{os.path.splitext(img_file)[0]}.json")

        # Copy image to output directory
        shutil.copy(img_path, images_dir)

        # Open image to get dimensions
        with Image.open(img_path) as img:
            width, height = img.size

        # Convert annotations to YOLO format and save
        yolo_annotations = convert_to_yolo_format(annotation_path, width, height)
        label_path = os.path.join(labels_dir, f"{os.path.splitext(img_file)[0]}.txt")
        with open(label_path, "w") as label_file:
            label_file.write("\n".join(yolo_annotations))

print(f"Dataset organized successfully!")
print(f"Train images: {len(train_files)}, Validation images: {len(val_files)}")